### Import Necessary Libraries

In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime

### Define Auxiliary Function

In [2]:
def getTickerPrice(ticker: str, date: datetime) -> float:
    return random.uniform(1, 100)

### Load the Data

In [52]:
df = pd.read_csv('testData.csv')
df.head(3)

,disclosureYear,disclosureDate,transactionDate,owner,ticker,assetDescription,type,amount,representative,district,capitalGainsOver200USD,option_symbol
0,2023,6/15/2023,5/20/2023,Spouse,AAPL,Apple Inc. Stock,Purchase,"$100,001 - $250,000",Nancy Pelosi,CA-12,Yes,NaN
1,2023,5/12/2023,4/10/2023,Self,GOOGL,Alphabet Inc. Stock,Sale (Full),"$50,001 - $100,000",Nancy Pelosi,CA-12,No,NaN
2,2023,12/29/2023,12/6/2022,Dependent,AMZN,AMZN Stock,Purchase,"$100,001 - $250,000",Nancy Pelosi,NY-14,Yes,NaN


In [74]:
df.columns

Index(['disclosureYear', 'disclosureDate', 'Date', 'owner', 'Symbol',
       'assetDescription', 'Side', 'Price', 'representative', 'district',
       'capitalGainsOver200USD', 'Size'],
      dtype='object')

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   disclosureYear          102 non-null    int64 
 1   disclosureDate          102 non-null    object
 2   transactionDate         102 non-null    object
 3   owner                   102 non-null    object
 4   ticker                  102 non-null    object
 5   assetDescription        102 non-null    object
 6   type                    102 non-null    object
 7   amount                  102 non-null    object
 8   representative          102 non-null    object
 9   district                102 non-null    object
 10  capitalGainsOver200USD  102 non-null    object
 11  option_symbol           2 non-null      object
dtypes: int64(1), object(11)
memory usage: 9.7+ KB


In [59]:
df.dtypes

disclosureYear             int64
disclosureDate            object
transactionDate           object
owner                     object
ticker                    object
assetDescription          object
type                      object
amount                    object
representative            object
district                  object
capitalGainsOver200USD    object
option_symbol             object
dtype: object

In [60]:
df['disclosureDate'] = pd.to_datetime(df['disclosureDate'], errors='coerce')
df['transactionDate'] = pd.to_datetime(df['transactionDate'], errors='coerce')

In [62]:
df.isnull().sum()

disclosureYear              0
disclosureDate              0
transactionDate             0
owner                       0
ticker                      0
assetDescription            0
type                        0
amount                      0
representative              0
district                    0
capitalGainsOver200USD      0
option_symbol             100
dtype: int64

In [63]:
df['type'].unique()

array(['Purchase', 'Sale (Full)', 'Sale (Partial)'], dtype=object)

In [64]:
df['ticker'].unique()

array(['AAPL', 'GOOGL', 'AMZN', 'MSFT', 'TSLA'], dtype=object)

### Preprocess the Data

In [65]:
# Drop the 'option_symbol' column
df = df.drop(columns=['option_symbol'])

In [66]:
# Ensure DataFrame matches required format
df.rename(columns={'transactionDate': 'Date', 'ticker': 'Symbol', 'type': 'Side', 'amount': 'Price'}, inplace=True)

In [68]:
# Convert 'Side' to 'buy' or 'sell'
df['Side'] = df['Side'].apply(lambda x: 'buy' if 'Purchase' in x else 'sell')

In [69]:
# Convert 'Price' to a float value (taking the lower bound of the range)
df['Price'] = df['Price'].apply(lambda x: float(x.strip('$').replace(',', '').split(' - ')[0]))

In [70]:
# Ensure 'Size' column exists and defaults to 1 if not provided
if 'Size' not in df.columns:
    df['Size'] = 1.0
else:
    df['Size'] = df['Size'].fillna(1)

In [80]:
df.head()

,disclosureYear,disclosureDate,Date,owner,Symbol,assetDescription,Side,Price,representative,district,capitalGainsOver200USD,Size
0,2023,2023-06-15,2023-05-20,Spouse,AAPL,Apple Inc. Stock,buy,100001.0,Nancy Pelosi,CA-12,Yes,1.0
1,2023,2023-05-12,2023-04-10,Self,GOOGL,Alphabet Inc. Stock,sell,50001.0,Nancy Pelosi,CA-12,No,1.0
2,2023,2023-12-29,2022-12-06,Dependent,AMZN,AMZN Stock,buy,100001.0,Nancy Pelosi,NY-14,Yes,1.0
3,2023,2022-02-13,2022-04-14,Self,AMZN,AMZN Stock,buy,100001.0,Nancy Pelosi,FL-9,No,1.0
4,2023,2023-06-23,2022-03-11,Joint,MSFT,MSFT Stock,buy,1001.0,Nancy Pelosi,CA-12,Yes,1.0


### Define Trade Performance Function

In [78]:
def calculate_trade_performance(df: pd.DataFrame) -> pd.Series:
    if df.empty:
        return pd.Series(dtype=float)

    # Calculate value of each trade
    df['Trade Value'] = df['Size'] * df['Price']

    # Calculate market price for each trade
    df['Market Price'] = df.apply(lambda row: getTickerPrice(row['Symbol'], row['Date']), axis=1)

    # Calculate unrealized P&L
    df['Unrealized P&L'] = df.apply(
        lambda row: (row['Market Price'] - row['Price']) * row['Size'] if row['Side'] == 'buy' else (row['Price'] - row['Market Price']) * row['Size'], axis=1)

    # Metrics calculation
    metrics = {}
    metrics['Total Trades'] = len(df)
    metrics['Total Buy Trades'] = len(df[df['Side'] == 'buy'])
    metrics['Total Sell Trades'] = len(df[df['Side'] == 'sell'])
    metrics['Total Volume'] = df['Size'].sum()
    metrics['Total Buy Volume'] = df[df['Side'] == 'buy']['Size'].sum()
    metrics['Total Sell Volume'] = df[df['Side'] == 'sell']['Size'].sum()
    metrics['Average Trade Value'] = df['Trade Value'].mean()
    metrics['Average Buy Trade Value'] = df[df['Side'] == 'buy']['Trade Value'].mean()
    metrics['Average Sell Trade Value'] = df[df['Side'] == 'sell']['Trade Value'].mean()
    metrics['Total Unrealized P&L'] = df['Unrealized P&L'].sum()

    return pd.Series(metrics)


### Apply the Trade Performance Calculation Function

In [81]:
metrics = calculate_trade_performance(df)
print(metrics)

Total Trades                1.020000e+02
Total Buy Trades            4.000000e+01
Total Sell Trades           6.200000e+01
Total Volume                1.020000e+02
Total Buy Volume            4.000000e+01
Total Sell Volume           6.200000e+01
Average Trade Value         8.036375e+04
Average Buy Trade Value     6.792600e+04
Average Sell Trade Value    8.838810e+04
Total Unrealized P&L        2.762547e+06
dtype: float64


In [82]:
df.head()

,disclosureYear,disclosureDate,Date,owner,Symbol,assetDescription,Side,Price,representative,district,capitalGainsOver200USD,Size,Trade Value,Market Price,Unrealized P&L
0,2023,2023-06-15,2023-05-20,Spouse,AAPL,Apple Inc. Stock,buy,100001.0,Nancy Pelosi,CA-12,Yes,1.0,100001.0,87.290559,-99913.709441
1,2023,2023-05-12,2023-04-10,Self,GOOGL,Alphabet Inc. Stock,sell,50001.0,Nancy Pelosi,CA-12,No,1.0,50001.0,32.841841,49968.158159
2,2023,2023-12-29,2022-12-06,Dependent,AMZN,AMZN Stock,buy,100001.0,Nancy Pelosi,NY-14,Yes,1.0,100001.0,1.930235,-99999.069765
3,2023,2022-02-13,2022-04-14,Self,AMZN,AMZN Stock,buy,100001.0,Nancy Pelosi,FL-9,No,1.0,100001.0,45.168175,-99955.831825
4,2023,2023-06-23,2022-03-11,Joint,MSFT,MSFT Stock,buy,1001.0,Nancy Pelosi,CA-12,Yes,1.0,1001.0,68.642837,-932.357163


### Interpretation of Results

In [83]:
for metric, value in metrics.items():
    print(f"{metric}: {value:.2f}")

Total Trades: 102.00
Total Buy Trades: 40.00
Total Sell Trades: 62.00
Total Volume: 102.00
Total Buy Volume: 40.00
Total Sell Volume: 62.00
Average Trade Value: 80363.75
Average Buy Trade Value: 67926.00
Average Sell Trade Value: 88388.10
Total Unrealized P&L: 2762546.79
